In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import parallel_where_binary, Aligner4
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon


In [2]:
from registration_utilities import affine_components_to_vector

In [3]:
stack_fixed = 'MD590'

In [4]:
train_sample_scheme = 1
global_transform_scheme = 1

stack_moving = 'atlas_on_MD589'

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [6]:
label_to_name_fixed = {i+1: name for i, name in enumerate(sorted(structures))}
name_to_label_fixed = {n:l for l, n in label_to_name_fixed.iteritems()}

In [7]:
label_to_name_moving = {i+1: name for i, name in enumerate(sorted(structures) + sorted([s+'_surround' for s in structures]))}
name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

In [8]:
volume_fixed = {name_to_label_fixed[name]: \
                DataManager.load_score_volume(stack_fixed, label=name, 
                                downscale=32, train_sample_scheme=train_sample_scheme)
               for name in structures}

print volume_fixed.values()[0].shape
print volume_fixed.values()[0].dtype

vol_fixed_xmin, vol_fixed_ymin, vol_fixed_zmin = (0,0,0)
vol_fixed_ymax, vol_fixed_xmax, vol_fixed_zmax = np.array(volume_fixed.values()[0].shape) - 1

(408, 548, 358)
float16


In [9]:
volume_moving = {name_to_label_moving[name]: DataManager.load_score_volume(stack_moving, label=name, downscale=32)
                 for name in structures + [s+'_surround' for s in structures]}

print volume_moving.values()[0].shape
print volume_moving.values()[0].dtype

vol_moving_xmin, vol_moving_ymin, vol_moving_zmin = (0,0,0)
vol_moving_ymax, vol_moving_xmax, vol_moving_zmax = np.array(volume_moving.values()[0].shape) - 1

(348, 421, 377)
float16


In [10]:
volume_moving_structure_sizes = {l: np.count_nonzero(vol > 0) for l, vol in volume_moving.iteritems()}

In [11]:
def convert_to_original_name(name):
    return name.split('_')[0]

labelIndexMap_m2f = {}
label_weights_m = {}
for label_m, name_m in label_to_name_moving.iteritems():
    labelIndexMap_m2f[label_m] = name_to_label_fixed[convert_to_original_name(name_m)]
    
    if 'surround' in name_m:
        label_weights_m[label_m] = 0
#         label_weights_m[label_m] = -np.minimum(1e5 / volume_moving_structure_sizes[label_m], 1.)
    else:
#         label_weights_m[label_m] = 1
        label_weights_m[label_m] = np.minimum(1e5 / volume_moving_structure_sizes[label_m], 1.)

In [12]:
# aligner = Aligner4(volume_fixed, volume_moving, labelIndexMap_m2f={14:14})

In [12]:
aligner = Aligner4(volume_fixed, volume_moving, labelIndexMap_m2f=labelIndexMap_m2f)

In [13]:
aligner.set_centroid(centroid_m='volume_centroid', centroid_f='volume_centroid')
# aligner.set_centroid(centroid_m='structure_centroid', centroid_f='centroid_m', indices_m=[name_to_label_moving['SNR_R']])

In [14]:
gradient_filepath_map_f = {ind_f: VOLUME_ROOTDIR + '/%(stack)s/score_volume_gradients/%(stack)s_down32_scoreVolume_%(label)s_trainSampleScheme_%(scheme)d_%%(suffix)s.bp' % \
                           {'stack': stack_fixed, 'label': label_to_name_fixed[ind_f], 'scheme':train_sample_scheme}
                           for ind_m, ind_f in labelIndexMap_m2f.iteritems()}

In [15]:
aligner.load_gradient(gradient_filepath_map_f=gradient_filepath_map_f, indices_f=None)

set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])


load gradient 1: 0.551081 seconds
load gradient 2: 0.866194 seconds
load gradient 3: 1.004477 seconds
load gradient 4: 0.852880 seconds
load gradient 5: 1.197654 seconds
load gradient 6: 0.707380 seconds
load gradient 7: 1.063113 seconds
load gradient 8: 0.877818 seconds
load gradient 9: 0.830057 seconds
load gradient 10: 1.008115 seconds
load gradient 11: 1.205821 seconds
load gradient 12: 0.841463 seconds
load gradient 13: 0.775160 seconds
load gradient 14: 1.246371 seconds
load gradient 15: 0.752250 seconds
load gradient 16: 1.009828 seconds
load gradient 17: 1.312571 seconds
load gradient 18: 1.098847 seconds
load gradient 19: 0.929699 seconds
load gradient 20: 1.203446 seconds
load gradient 21: 1.042641 seconds
load gradient 22: 1.038572 seconds
load gradient 23: 1.112002 seconds
load gradient 24: 1.246246 seconds
load gradient 25: 0.672761 seconds
load gradient 26: 0.879407 seconds
load gradient 27: 0.611480 seconds
load gradient 28: 1.278779 seconds
overall: 27.220286 seconds


In [16]:
# largely the same optimization path regardless of the starting condition

# For rigid, 
# grad_computation_sample_number = 1e5 is desired
# grid_search_iteration_number and grid_search_sample_number seem to be unimportant as well, set to 100
# lr1=10, lr2=.1 is best

# For affine, 
# lr2 = .001 is too slow; 0.1 rises faster than 0.01
# lr1 does not matter
# plateus around iteration 100, but keep rising afterwards.
# grad_computation_sample_number does not make a difference

while True:

    try:

        if global_transform_scheme == 1:

            T, scores = aligner.optimize(type='affine', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
#                                          indices_m=[name_to_label_moving['SC'], name_to_label_moving['IC'],
#                                                    name_to_label_moving['SC_surround'], name_to_label_moving['IC_surround']],
                                         indices_m=None,
                                        grid_search_iteration_number=30,
                                         grid_search_sample_number=100,
                                         grad_computation_sample_number=1e5,
                                         lr1=10, lr2=0.1,
                                         label_weights=label_weights_m,
                                        std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))

        elif global_transform_scheme == 2:

            T, scores = aligner.optimize(type='rigid', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
                                         indices_m=None,
                                        grid_search_iteration_number=30,
                                         grid_search_sample_number=100,
                                         grad_computation_sample_number=1e5,
                                         lr1=10, lr2=0.1,
                                        label_weights=label_weights_m,
                                        std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))
        
        break
        
    except Exception as e:
        sys.stderr.write(e.message + '\n')


grid search: 16.709428 seconds
-inf 0.672978
grid search: 10.387258 seconds
0.672978 0.690231
grid search: 8.980313 seconds
0.690231 0.716411
grid search: 6.971288 seconds
0.716411 0.735678
grid search: 4.875279 seconds
0.735678 0.751332
grid search: 3.770371 seconds
grid search: 2.660933 seconds
grid search: 2.566297 seconds
0.751332 0.757350
grid search: 1.555729 seconds
grid search: 1.561698 seconds
grid search: 1.453049 seconds
grid search: 1.456872 seconds
grid search: 1.363973 seconds
0.757350 0.759122
grid search: 1.348753 seconds
grid search: 1.351601 seconds
grid search: 1.360196 seconds
grid search: 1.346268 seconds
grid search: 1.365255 seconds
grid search: 1.349464 seconds
grid search: 1.361879 seconds
grid search: 1.356725 seconds
grid search: 1.354583 seconds
grid search: 1.344376 seconds
grid search: 1.352803 seconds
grid search: 1.349171 seconds
grid search: 1.361677 seconds
grid search: 1.350581 seconds
grid search: 1.360458 seconds
grid search: 1.357703 seconds
grid s

params_best_upToNow [-6.8041999   7.00410767  0.5067157  -0.19106513]


score: 0.759122
iteration 1
score: 0.607479
iteration 2
score: 0.553549
iteration 3
score: 0.791204
iteration 4
score: 0.705156
iteration 5
score: 0.836890
iteration 6
score: 0.819419
iteration 7
score: 0.844245
iteration 8
score: 0.868755
iteration 9
score: 0.881066
iteration 10
score: 0.903665
iteration 11
score: 0.903372
iteration 12
score: 0.926684
iteration 13
score: 0.919039
iteration 14
score: 0.942722
iteration 15
score: 0.931808
iteration 16
score: 0.953625
iteration 17
score: 0.944341
iteration 18
score: 0.962038
iteration 19
score: 0.953141
iteration 20
score: 0.967754
iteration 21
score: 0.960942
iteration 22
score: 0.971211
iteration 23
score: 0.964839
iteration 24
score: 0.969879
iteration 25
score: 0.965556
iteration 26
score: 0.968356
iteration 27
score: 0.967032
iteration 28
score: 0.966485
iteration 29
score: 0.970606
iteration 30
score: 0.970886
iteration 31
score: 0.975058
iteration 32
score: 0.975353
iteration 33
score: 0.977958
iteration 34
score: 0.978526
iterati

KeyboardInterrupt: 

In [17]:
print T.reshape((3,4))
plt.plot(scores);
print max(scores), scores[-1]

NameError: name 'T' is not defined

In [132]:
# # Consistency analysis

# trial_num = 3
# trial_results = []

# for _ in range(trial_num):

#     if global_transform_scheme == 1:

#         T, scores = aligner.optimize(type='affine', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
#                                      indices_m=None,
#                                     grid_search_iteration_number=30,
#                                      grid_search_sample_number=100,
#                                      grad_computation_sample_number=1e5,
#                                      lr1=10, lr2=0.1,
#                                     label_weights=label_weights_m,
#                                 std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))

#     elif global_transform_scheme == 2:

#         T, scores = aligner.optimize(type='rigid', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
# #                                      indices_m=[name_to_label_moving['Tz'], 
# #                                             name_to_label_moving['12N'], 
# #                                             name_to_label_moving['Pn']],
#                                      indices_m=None,
#                                     grid_search_iteration_number=30,
#                                      grid_search_sample_number=100,
#                                      grad_computation_sample_number=1e5,
#                                      lr1=10, lr2=0.1,
#                                     label_weights=label_weights_m,
#                                 std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))


#     print T.reshape((3,4))
# #     plt.plot(scores);
# #     plt.show();
#     print max(scores), scores[-1]
    
#     trial_results.append((scores, T))

In [133]:
# fig, axes = plt.subplots(5, 2, figsize=(10,10), sharey=True, sharex=True)
# axes = axes.flatten()
# for i, (res, t) in enumerate(trial_results):
#     axes[i].plot(res)
# plt.tight_layout()
# plt.show()

In [18]:
# atlasAlignParams_dir = create_if_not_exists(atlasAlignParams_rootdir + '/%(stack_moving)s_to_%(stack_fixed)s' % \
#                      {'stack_moving': stack_moving, 'stack_fixed': stack_fixed})

In [27]:
params_fp = DataManager.get_global_alignment_parameters_filepath(stack_moving=stack_moving,
                                                                stack_fixed=stack_fixed,
                                                                train_sample_scheme=train_sample_scheme,
                                                                global_transform_scheme=global_transform_scheme)


DataManager.save_alignment_parameters(params_fp, T, 
                                      aligner.centroid_m, aligner.centroid_f,
                                      aligner.xdim_m, aligner.ydim_m, aligner.zdim_m, 
                                      aligner.xdim_f, aligner.ydim_f, aligner.zdim_f)

# with open(atlasAlignParams_dir + '/%(stack_moving)s_down32_scoreVolume_to_%(stack_fixed)s_down32_scoreVolume_trainSampleScheme_%(scheme)d_globalTxScheme_%(gtf_sheme)d_parameters.txt' % \
#           {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'scheme':train_sample_scheme, 'gtf_sheme':global_transform_scheme}, 'w') as f:
# with open(params_fp, 'w') as f:
    
#     f.write(array_to_one_liner(T))
#     f.write(array_to_one_liner(aligner.centroid_m))
#     f.write(array_to_one_liner([aligner.xdim_m, aligner.ydim_m, aligner.zdim_m]))
#     f.write(array_to_one_liner(aligner.centroid_f))
#     f.write(array_to_one_liner([aligner.xdim_f, aligner.ydim_f, aligner.zdim_f]))

In [28]:
score_plot_fp = DataManager.get_global_alignment_score_plot_filepath(stack_moving=stack_moving,
                                                                    stack_fixed=stack_fixed,
                                                                    train_sample_scheme=train_sample_scheme,
                                                                    global_transform_scheme=global_transform_scheme)

fig = plt.figure();
plt.plot(scores);
plt.savefig(score_plot_fp, bbox_inches='tight')
plt.close(fig)

In [20]:
# Try all sets of parameters 

In [21]:
# output_dir = create_if_not_exists(atlasAlignParams_dir + '/rigid_score_plot')

In [22]:
# c = 0

# for lr1 in [.1, 1., 10.]:
#     for lr2 in [1e-1, 1e-2, 1e-3]:
#         for n in [1e3, 1e4, 1e5]:
            
            
#             c += 1
        
#             T, scores = aligner.optimize(type='rigid', max_iter_num=500, history_len=50, terminate_thresh=1e-4,
#             #                              indices_m=[name_to_label_fixed['SNR_L']],
#                                          indices_m=None,
#                                         grid_search_iteration_number=10,
#                                          grid_search_sample_number=100,
#                                          grad_computation_sample_number=n,
#                                          lr1=lr1, lr2=lr2)
#             #                              init_T=[1,0,0,-10,0,1,0,0,0,0,1,0])
            
#             plt.figure();
#             plt.plot(scores);
#             plt.savefig(atlasAlignParams_dir + '/rigid_score_plot/%(stack_moving)s_to_%(stack_fixed)s_scoreEvolution_parameterSet_%(c)d.png' % \
#                         {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'c':c},
#                         bbox_inches='tight')
            
#             with open(atlasAlignParams_dir + '/rigid_score_plot/%(stack_moving)s_to_%(stack_fixed)s_scoreEvolution_parameterSet_%(c)d.txt' % \
#                        {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'c':c}, 'w') as f:
#                 f.write(array_to_one_liner([lr1, lr2, n]))
# #             print max(scores), scores[-1]

In [23]:
# c = 0
# for lr1 in [.1, 1., 10.]:
#     for lr2 in [1e-1, 1e-2, 1e-3]:
#         for n in [1e3, 1e4, 1e5]:
#             c += 1
#             print c, lr1, lr2, int(n)

In [24]:
# READ

In [12]:
# stack_fixed = 'MD591'
stack_moving = 'atlas_on_MD589'

In [13]:
stacks_annotation = ['MD589', 'MD594']
# stack_annotation = 'MD589'

In [14]:
global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_global_alignment_parameters(stack_moving=stack_moving,
                                                    stack_fixed=stack_fixed,
                                                    train_sample_scheme=train_sample_scheme,
                                                    global_transform_scheme=global_transform_scheme)

In [15]:
volumes_annotation = {'MD594':bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_atlasAlignParams_atlas_v2/MD594_to_MD589/MD594_down32_annotationVolume_alignedTo_MD589_down32_annotationVolume.bp'),
                      'MD589': bp.unpack_ndarray_file(VOLUME_ROOTDIR + '/MD589/MD589_down32_annotationVolume.bp')}

In [16]:
from registration_utilities import transform_volume

In [17]:
annotation_volumes_volume_m_aligned_to_f = {}

for stack, volume_annotation in volumes_annotation.iteritems():
    
    annotation_volumes_volume_m_aligned_to_f[stack] = transform_volume(vol=volume_annotation, 
                                                                       global_params=global_params, 
                                                                       centroid_m=centroid_m, 
                                                                       centroid_f=centroid_f,
                                                                      xdim_f=xdim_f,
                                                                      ydim_f=ydim_f,
                                                                      zdim_f=zdim_f)
    
    output_fn = DataManager.get_transformed_volume_filepath(stack_m=stack, type_m='annotation',
                                                stack_f=stack_fixed, type_f='score',
                                                downscale=32, train_sample_scheme_f=train_sample_scheme)
    
    create_if_not_exists(os.path.dirname(output_fn))

    bp.pack_ndarray_file(annotation_volumes_volume_m_aligned_to_f[stack], output_fn)

In [18]:
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = np.loadtxt('/home/yuncong/csd395/CSHL_volumes2/%(stack_fixed)s/score_volumes/%(stack_fixed)s_down32_scoreVolume_7N_bbox.txt' %\
          dict(stack_fixed=stack_fixed)).astype(np.int)
print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

0 547 0 407 110 467


In [37]:
from registration_utilities import find_contour_points

In [38]:
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

In [39]:
# viz_dir = create_if_not_exists(atlasAlignParams_dir + '/%(stack_moving)s_down32_scoreVolume_to_%(stack_fixed)s_down32_scoreVolume_trainSampleScheme_%(scheme)d_global_viz' % \
#           {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'scheme':train_sample_scheme})

viz_dir = create_if_not_exists(DataManager.get_global_alignment_viz_filepath(stack_moving=stack_moving,
                                                        stack_fixed=stack_fixed,
                                                        train_sample_scheme=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme))

In [40]:
first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]

In [41]:
stack_colors = {'MD589': (255,0,0), 'MD594': (0,255,0)}

In [42]:
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1):
    
    if metadata_cache['sections_to_filenames'][stack_fixed][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
        continue
    
    bar.value = sec

    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    
    # Find fixed volume annotation contours
#     contours_f_on_volume = find_contour_points(volume_fixed[..., int(z)])
#     contours_f_on_cropped = {i: [cnt + (xmin_vol_f, ymin_vol_f) for cnt in cnts] for i, cnts in contours_f_on_volume.iteritems()}

    # Find moving volume annotation contours
    
    for stack, volume_m_aligned_to_f in annotation_volumes_volume_m_aligned_to_f.iteritems():
        contours_m_alignedTo_f_on_volume = find_contour_points(volume_m_aligned_to_f[..., int(z)])
        contours_m_alignedTo_f_on_cropped = {i: [cnt + (xmin_vol_f, ymin_vol_f) for cnt in cnts] 
                                             for i, cnts in contours_m_alignedTo_f_on_volume.iteritems()}

    #     # Draw fixed volume annotation contours
    #     for ind_f, cnts_f in contours_f_on_cropped.iteritems():
    #         for cnt_f in cnts_f:
    #             cv2.polylines(viz, [cnt_f.astype(np.int)], True, (0,255,0), 2)

        # Draw moving volume annotation contours
        for ind_m, cnts_m in contours_m_alignedTo_f_on_cropped.iteritems():
            for cnt_m in cnts_m:
                cv2.polylines(viz, [cnt_m.astype(np.int)], True, stack_colors[stack], 2)

    viz_fn = os.path.join(viz_dir, '%(stack_moving)s_to_%(stack_fixed)s_%(sec)04d.jpg' % \
          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'sec': sec})
    imsave(viz_fn, viz)

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


In [19]:
from registration_utilities import transform_volume

In [24]:
# v = DataManager.load_score_volume(stack='atlas_on_MD589', label='7N_L', downscale=32)

# plt.imshow(v[:,:,120]);
# plt.colorbar();
# plt.show();

# plt.imshow(v[:,:,270]);
# plt.colorbar();
# plt.show();

In [25]:
# Transform moving volume, sided

structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

for name_s in structures_sided:
    
    print name_s
    
    vol_m = DataManager.load_score_volume(stack=stack_moving, label=name_s, downscale=32)
    
    volume_m_alignedTo_f = \
    transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                      xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)
    
    volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                            stack_f=stack_fixed, type_f='score',
                                            label=name_s,
                                            downscale=32, 
                                            train_sample_scheme_f=1)
    
    create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
    
    bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

5N_L
5N_R
6N_L
6N_R
7N_L
7N_R
7n_L
7n_R
Amb_L
Amb_R
LC_L
LC_R
LRt_L
LRt_R
Pn_L
Pn_R
Tz_L
Tz_R
VLL_L
VLL_R
RMC_L
RMC_R
SNC_L
SNC_R
SNR_L
SNR_R
3N_L
3N_R
4N_L
4N_R
Sp5I_L
Sp5I_R
Sp5O_L
Sp5O_R
Sp5C_L
Sp5C_R
PBG_L
PBG_R
10N_L
10N_R
VCA_L
VCA_R
VCP_L
VCP_R
DC_L
DC_R
AP
12N
RtTg
SC
IC


In [20]:
# Transform moving volume, sided

structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

for name_s in structures_sided:
    
    print name_s+'_surround'
    
    vol_m = DataManager.load_score_volume(stack=stack_moving, label=name_s+'_surround', downscale=32)
    
    volume_m_alignedTo_f = \
    transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                      xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)
    
    volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                            stack_f=stack_fixed, type_f='score',
                                            label=name_s+'_surround',
                                            downscale=32, 
                                            train_sample_scheme_f=1)
    
    create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
    
    bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

5N_L
5N_R
6N_L
6N_R
7N_L
7N_R
7n_L
7n_R
Amb_L
Amb_R
LC_L
LC_R
LRt_L
LRt_R
Pn_L
Pn_R
Tz_L
Tz_R
VLL_L
VLL_R
RMC_L
RMC_R
SNC_L
SNC_R
SNR_L
SNR_R
3N_L
3N_R
4N_L
4N_R
Sp5I_L
Sp5I_R
Sp5O_L
Sp5O_R
Sp5C_L
Sp5C_R
PBG_L
PBG_R
10N_L
10N_R
VCA_L
VCA_R
VCP_L
VCP_R
DC_L
DC_R
AP
12N
RtTg
SC
IC
